### Import data

In [1]:
import pandas as pd
df = pd.read_csv('./data/KPM_qwen_in_1by1.csv')
print(df.columns)

Index(['topic', 'stance', 'argument', 'matched_kp(group)',
       'confidence_score(group)', 'matched_kp(1by1)', 'confidence_score(1by1)',
       'matched_kp(1by1))'],
      dtype='object')


In [2]:
prediction_df = pd.read_csv('./data/qwen_1by1+predictions_in_group.csv')
print(prediction_df.columns)

Index(['topic', 'stance', 'arguments', 'kep_points',
       'predict_kps(avg_embedding)', 'predict_kps(best_embedding)',
       'predict_kps(PCA_embedding)'],
      dtype='object')


### Calculate confidence score one by one and get best one

In [3]:
import random
from http import HTTPStatus
import dashscope
import re
import yaml

In [ ]:
with open("../conf/index.yaml") as f:
    credentials = yaml.safe_load(f)
qwen_api_key = credentials['environment_variables']['QWEN_DASHSCOPE_API_KEY']

In [4]:
def generating(topic, argument,kp):
    try:
        messages = [
            {
                'role': 'system',
                'content': f"""
            You need to evaluate to what extend the 2 sentences matches to each other based on the topic "{topic}" and return a confidence score between 0 and 1. 0 represents totally not match, 1 represents very match. Your return should be in this format:Confidence Score: number
            """
            },
            {
                'role': 'user',
                'content': "argument:"+argument+", kp:"+kp
            }
        ]
        dashscope.api_key = qwen_api_key
        response = dashscope.Generation.call(model="qwen-max",
                                   messages=messages,
                                   # 设置随机数种子seed，如果没有设置，则随机数种子默认为1234
                                   seed=random.randint(1, 10000),
                                   # 将输出设置为"message"格式
                                   result_format='message')
        if response.status_code == HTTPStatus.OK:
            output = response['output']["choices"][0]["message"]["content"]
            print(response['output']["choices"][0]["message"]["content"])
            # confidence_score_match = re.findall(r'Confidence Score: (\d+\.\d+)', output)
            confidence_score_match = re.findall(r'Confidence Score: (\d+(\.\d+)?)', output)

            if confidence_score_match:
                print(confidence_score_match)
                score = float(confidence_score_match[0][0])
                print("KP score: "+str(score))
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
                return score
            else:
                print(f"Error. Regenerating string...")
                return generating(topic, argument, kp)
        else:
            print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message
            ))
            return generating(topic, argument, kp)
    except Exception as e:
        print(f"An error occurred: {e}. Retrying...")
        return generating(topic, argument, kp)

In [5]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calculate_bert_similarity(sentence1, sentence2):
    # Load pre-trained BERT model and tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Tokenize and obtain embeddings for each sentence
    tokens1 = tokenizer(sentence1, return_tensors='pt')
    tokens2 = tokenizer(sentence2, return_tensors='pt')

    with torch.no_grad():
        outputs1 = model(**tokens1)
        outputs2 = model(**tokens2)

    # Extract the embeddings from the model outputs
    embedding1 = outputs1.last_hidden_state.mean(dim=1).numpy()
    embedding2 = outputs2.last_hidden_state.mean(dim=1).numpy()

    # Reshape embeddings for cosine similarity calculation
    embedding1 = np.squeeze(embedding1)
    embedding2 = np.squeeze(embedding2)

    # Calculate cosine similarity
    similarity_score = cosine_similarity([embedding1], [embedding2])[0][0]

    return similarity_score

In [17]:
import ast
for index,row in df.iloc[7000:].iterrows():
    topic = row['topic']
    stance = row['stance']
    argument = row['argument']
    filtered_df = prediction_df[(prediction_df['topic']==topic) & (prediction_df['stance']== stance)]
    # print(filtered_df)
    candidate_kps = filtered_df['predict_kps(avg_embedding)'].values.tolist()
    candidate_kps = ast.literal_eval(candidate_kps[0])
    print(index)

    best_kp = ''
    confidence_score = 0.0
    for kp in candidate_kps:
        print(kp)
        new_score = generating(topic, argument, kp)
        if new_score >= confidence_score:
            if new_score == confidence_score:
                old_cos = calculate_bert_similarity(argument, best_kp)
                new_cos = calculate_bert_similarity(argument, kp)
                if new_cos>old_cos:
                    best_kp = kp
                    confidence_score = new_score
            else:
                best_kp = kp
                confidence_score = new_score
    print("Best KP: "+best_kp)
    print("Confidence Score: "+ str(confidence_score))
    df.at[index,'matched_kp(1by1)'] = best_kp
    df.at[index,'confidence_score(1by1)'] = confidence_score

7000
Opposing government regulation of social media due to potential privacy infringement for users.
Confidence Score: 1
[('1', '')]
KP score: 1.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Individual social networks should self-regulate rather than relying on government intervention.
Confidence Score: 0.8
[('0.8', '.8')]
KP score: 0.8
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Government regulation of social media risks being used to suppress dissent and undermine free speech.
Confidence Score: 1
[('1', '')]
KP score: 1.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


G:\Program Files\KPA prompt-based\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of

Government regulation of social media platforms may lead to potential misuse and infringement of user autonomy.
Confidence Score: 0.8
[('0.8', '.8')]
KP score: 0.8
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Government regulation of social media platforms is impractical due to the emergence of alternative, uncontrollable platforms.
Confidence Score: 0.8
[('0.8', '.8')]
KP score: 0.8
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Best KP: Government regulation of social media risks being used to suppress dissent and undermine free speech.
Confidence Score: 1.0
7001
Opposing government regulation of social media due to potential privacy infringement for users.
Confidence Score: 0.75
[('0.75', '.75')]
KP score: 0.75
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Individual social networks should self-regulate rather than relying on government intervention.
Confidence Score: 0.75
[('0.75', '.75')]
KP score: 0.75
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


G:\Program Files\KPA prompt-based\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of

Government regulation of social media risks being used to suppress dissent and undermine free speech.
Confidence Score: 1
[('1', '')]
KP score: 1.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Government regulation of social media platforms may lead to potential misuse and infringement of user autonomy.
Confidence Score: 0.85
[('0.85', '.85')]
KP score: 0.85
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Government regulation of social media platforms is impractical due to the emergence of alternative, uncontrollable platforms.
Confidence Score: 0.5
[('0.5', '.5')]
KP score: 0.5
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Best KP: Government regulation of social media risks being used to suppress dissent and undermine free speech.
Confidence Score: 1.0
7002
Opposing government regulation of social media due to potential privacy infringement for users.
Confidence Score: 1
[('1', '')]
KP score: 1.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Individual social networks should self-regulate rather tha

...

Best KP: High taxes in the USA negatively impact the desirability of living there for some individuals.
Confidence Score: 1.0
7235
Racism prevalent in the USA hampers social harmony.
Confidence Score: 1
[('1', '')]
KP score: 1.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
High taxes in the USA negatively impact the desirability of living there for some individuals.
Confidence Score: 0.5
[('0.5', '.5')]
KP score: 0.5
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The USA's fast-paced lifestyle can be highly stressful.
Confidence Score: 0.2
[('0.2', '.2')]
KP score: 0.2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The USA's healthcare system is criticized for its high costs and bias against the economically disadvantaged.
Confidence Score: 0.6
[('0.6', '.6')]
KP score: 0.6
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The high cost of living and expensive healthcare in the USA pose significant challenges for residents.
Confidence Score: 0.2
[('0.2', '.2')]
KP score: 0.2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

G:\Program Files\KPA prompt-based\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of

The USA's healthcare system is criticized for its high costs and bias against the economically disadvantaged.
Confidence Score: 0.2
[('0.2', '.2')]
KP score: 0.2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The high cost of living and expensive healthcare in the USA pose significant challenges for residents.
Confidence Score: 0.5
[('0.5', '.5')]
KP score: 0.5
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


G:\Program Files\KPA prompt-based\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of

Best KP: The USA's fast-paced lifestyle can be highly stressful.
Confidence Score: 0.5
7237
Racism prevalent in the USA hampers social harmony.
Confidence Score: 0.2
[('0.2', '.2')]
KP score: 0.2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
High taxes in the USA negatively impact the desirability of living there for some individuals.
Confidence Score: 0.7
[('0.7', '.7')]
KP score: 0.7
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The USA's fast-paced lifestyle can be highly stressful.
Confidence Score: 0.2
[('0.2', '.2')]
KP score: 0.2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The USA's healthcare system is criticized for its high costs and bias against the economically disadvantaged.
Confidence Score: 0.5
[('0.5', '.5')]
KP score: 0.5
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The high cost of living and expensive healthcare in the USA pose significant challenges for residents.
Confidence Score: 0.5
[('0.5', '.5')]
KP score: 0.5
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Best KP: High taxes in

In [18]:
df.to_csv('./data/KPM_qwen_in_1by1.csv', index=False)
print("Add new data successfully!!!!!")


Add new data successfully!!!!!


In [1]:
import pandas as pd
df = pd.read_csv('./data/KPM_qwen_in_1by1.csv')
print(df.columns)

Index(['topic', 'stance', 'argument', 'matched_kp(group)',
       'confidence_score(group)', 'matched_kp(1by1)',
       'confidence_score(1by1)'],
      dtype='object')


In [3]:
score_list = df['confidence_score(1by1)'].values.tolist()
print(sum(score_list)/len(score_list))

0.952727272727263
